# Generate Documents - Svenska Kyrkan i Wien

In [ ]:
# import sys
# !python -m pip install openpyxl

In [ ]:
# pd.set_option('display.max_rows', 100)

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

In [ ]:
from openpyxl import Workbook, load_workbook
import shutil
from copy import copy

In [ ]:
from datetime import datetime

In [ ]:
BASE_DIR = Path.cwd()
print(BASE_DIR)

### Filnamn

Här kan man ändra namnen på alla filer och tabellblad för data som läses in ock skrivs ut

In [ ]:
inputDirectory = 'input_pask'
outputDirectory = 'output_pask'
templateDirectory = 'templates'

# Inputs
fileNameProducts = 'wc-product-export.csv'
fileNameOrders = 'orders.xlsx'
fileNameTimeslots = 'tidsbokning.csv'
fileNameProductList = 'product_list.xlsx'
sheetNamePackingCategories = 'packningskategorier'
fileNameSchedule = 'schedule.xlsx'

# Outputs
fileNamePackList = 'pack_list.xlsx'
fileNamePickUpList = 'pick_up_list.xlsx'
fileNameSemleList = 'semlelista.xlsx'
fileNameMasterList = 'masterlista.xlsx'
fileNameDiffs = 'diffs.xlsx'
fileNameAllData = 'allData.xlsx'
fileNameTimeslotsOutput = 'tidsbokning.xlsx'

# Templates
fileNamePackListTemplate = 'pack_list_template.xlsx'
fileNamePickUpListTemplate = 'pick_up_template.xlsx'
fileNameSemleListTemplate = 'semlelist_template.xlsx'

### Read and Format Data

In [ ]:
dfProducts = pd.read_csv(BASE_DIR / inputDirectory / fileNameProducts)

Harmonisera namnen mellan julbasar och påskbasar

In [ ]:
if 'Artikelnummer' in dfProducts.columns:
    dfProducts.rename(columns={'Artikelnummer':'SKU'}, inplace=True)
    
if 'Typ' in dfProducts.columns:
    dfProducts.rename(columns={'Typ':'Type'}, inplace=True)
    
if 'Regulärer Preis' in dfProducts.columns:
    dfProducts.rename(columns={'Regulärer Preis':'Regular price'}, inplace=True)

In [ ]:
dfOrders = pd.read_excel(str(BASE_DIR / inputDirectory / fileNameOrders))

Harmonisera namnen mellan julbasar och påskbasar

In [ ]:
if 'E-Mail-Adresse' in dfOrders.columns:
    dfOrders.rename(columns={'E-Mail-Adresse':'Email (Billing)'}, inplace=True)
    
if 'Telefon' in dfOrders.columns:
    dfOrders.rename(columns={'Telefon':'Telephone'}, inplace=True)
    
if 'Phone (Billing)' in dfOrders.columns:
    dfOrders.rename(columns={'Phone (Billing)':'Telephone'}, inplace=True)
    
if 'Info_OK' in dfOrders.columns:
    dfOrders.rename(columns={'Info_OK':'Info'}, inplace=True)

In [ ]:
dfTimeslots = pd.read_csv(BASE_DIR / inputDirectory / fileNameTimeslots, index_col=False,
                         parse_dates=['Time', 'app_date_1', 'app_datetime_1'], dayfirst=True)
#encoding='latin_1'

In [ ]:
# Timeslot list has a lot of peculiarities
dfTimeslots['Name:'] = dfTimeslots['Telefon']
dfTimeslots['Telefon'] = dfTimeslots['Unnamed: 21']
dfTimeslots.drop(columns='Unnamed: 21', inplace=True)

In [ ]:
# change pick-up time from string to real time format
dfTimeslots['app_slot_1'] = dfTimeslots['app_datetime_1'].transform(lambda x: x.time())

In [ ]:
dfPackingCategories = pd.read_excel(str(BASE_DIR / inputDirectory / fileNameProductList), sheet_name=sheetNamePackingCategories)

In [ ]:
dfSchedule = pd.read_excel(str(BASE_DIR / inputDirectory / fileNameSchedule))

Skapa en extra spalter med date och datetime från dag och tid

In [ ]:
dfSchedule['Date'] = dfSchedule['Dag'].transform(lambda x: x.date())

In [ ]:
dfSchedule['DateTime'] = [datetime.combine(d, t) for d, t in zip(dfSchedule['Dag'], dfSchedule['Tid'])]

**Re-Exportera Timeslots**

In [ ]:
dfTimeslots.to_excel(BASE_DIR / outputDirectory / fileNameTimeslotsOutput)

### Diffs

In [ ]:
def alignCellWidthFor(sheet):
    dims = {}
    for row in sheet.rows:
        for cell in row:
            if cell.value:
                dims[cell.column_letter] = max((dims.get(cell.column_letter, 0), len(str(cell.value))))    
    for col, value in dims.items():
        sheet.column_dimensions[col].width = value * 1.23

In [ ]:
Path.unlink(BASE_DIR / outputDirectory / fileNameDiffs, missing_ok=True)

**Har gett order men har ej bokat tid**

In [ ]:
orderEmails = dfOrders['Email (Billing)'].unique()

In [ ]:
timeSlotEmails = dfTimeslots['Email'].unique()

In [ ]:
diffEmails = np.setdiff1d(orderEmails, timeSlotEmails)
dfOrderButNoTimeBooked = dfOrders[dfOrders['Email (Billing)'].isin(diffEmails)].drop_duplicates(subset=['Order Number'])[['Order Number', 'Full Name (Billing)', 'Email (Billing)',
       'Order Total Amount', 'Order Total Amount (- Refund)', 'Order Date', 'Customer Note', 'Telephone']]

In [ ]:
dfOrderButNoTimeBooked.to_excel(BASE_DIR / outputDirectory / fileNameDiffs, sheet_name='Vi har inte bokat')

In [ ]:
workbook = load_workbook(filename=BASE_DIR / outputDirectory / fileNameDiffs)
sheet = workbook['Vi har inte bokat']
alignCellWidthFor(sheet)
workbook.save(filename=BASE_DIR / outputDirectory / fileNameDiffs)

**Dubbla tidsbokningar**

In [ ]:
emailCounts = dfTimeslots['Email'].value_counts()
dfMoreThanOneBookings = dfTimeslots[dfTimeslots['Email'].isin(emailCounts.index[emailCounts.gt(1)])].sort_values(by=['Email', 'app_date_1', 'app_slot_1'])
dfMoreThanOneBookings = dfMoreThanOneBookings[['Email', 'app_date_1', 'app_slot_1', 'Name:', 'Telefon', 'app_status_1']]

In [ ]:
with pd.ExcelWriter(BASE_DIR / outputDirectory / fileNameDiffs,
                    mode='a') as writer:  
    dfMoreThanOneBookings.to_excel(writer, sheet_name='Dubbla tidsbokningar')

In [ ]:
workbook = load_workbook(filename=BASE_DIR / outputDirectory / fileNameDiffs)
sheet = workbook['Dubbla tidsbokningar']
alignCellWidthFor(sheet)
workbook.save(filename=BASE_DIR / outputDirectory / fileNameDiffs)

In [ ]:
# after reporting, we remove the duplicates
dfTimeslots.drop_duplicates(subset=['Email'], inplace=True)

**Artikel saknas i produktplaceringslista**

In [ ]:
orderSKUs = dfOrders['SKU'].unique()

In [ ]:
produktListaArtiklar = dfProducts['SKU'].unique()

In [ ]:
skuDiff = np.setdiff1d(orderSKUs, produktListaArtiklar)

In [ ]:
missingProducts = dfOrders[dfOrders['SKU'].isin(skuDiff)][['SKU', 'Product Name']].drop_duplicates(subset=['SKU']).sort_values(by='SKU')
print(len(missingProducts))

In [ ]:
with pd.ExcelWriter(BASE_DIR / outputDirectory / fileNameDiffs,
                    mode='a') as writer:  
    missingProducts.to_excel(writer, sheet_name='Saknas i produktlista')

In [ ]:
workbook = load_workbook(filename=BASE_DIR / outputDirectory / fileNameDiffs)
sheet = workbook['Saknas i produktlista']
alignCellWidthFor(sheet)
workbook.save(filename=BASE_DIR / outputDirectory / fileNameDiffs)

### Merge and Select Data

In [ ]:
# Merge orders with timeslots
dfAllData = pd.merge(dfOrders, dfTimeslots, left_on='Email (Billing)', right_on='Email')

In [ ]:
# Merge with products
dfAllData = pd.merge(dfAllData, dfProducts, on='SKU')

In [ ]:
# Merge with product placements
# dfAllData = pd.merge(dfAllData, dfProductList, left_on='SKU', right_on='Artikelnummer', suffixes=[None, '_y'])

In [ ]:
dfAllData = pd.merge(dfAllData, dfPackingCategories, left_on='pa_packn_kategori', right_on='Packningskategori')

In [ ]:
# Fields with data
dfAllData = dfAllData[['Order Number', 'Full Name (Billing)', 'Email (Billing)',
                       'Order Total Amount (- Refund)', 'Order Date', 'Customer Note',
                       'Telephone', 'Quantity (- Refund)', 'Product Name', 'Item Cost (inc. tax)',
                       'Item #', 'Info', 'Time', 'app_date_1', 'app_slot_1', 'app_datetime_1',
                       'Type', 'SKU', 'Short description', 'Name',  'Regular price',
                       'Packningskategori', 'info till Packlistan']]

In [ ]:
# Standardized column namings
dfAllData.columns = ['OrderNumber', 'FullName', 'Email',
                     'OrderTotalAmount', 'OrderDate', 'CustomerNote',
                     'Telephone', 'Quantity', 'ProductName', 'ItemCost',
                     'ItemID', 'Info', 'OrderTime', 'PickUpDate', 'PickUpTime', 'PickUpDateTime',
                     'Type', 'ArticleNumber', 'ProductNameDE',
                     'Description', 'RegularPrice',
                     'PackingCategory', 'PickUpArea']

In [ ]:
# Selection of relevant data and more logical sorting of columns
dfAllData = dfAllData[['OrderNumber', 'OrderDate', 'OrderTime', 
                       'FullName', 'Email', 'Telephone', 'CustomerNote',
                       'ItemID', 'Info', 'Type', 'ArticleNumber', 'ProductName', 'ProductNameDE',
                       'Quantity',  'ItemCost', 'OrderTotalAmount', 'RegularPrice', 
                       'PickUpDate', 'PickUpTime', 'PickUpDateTime',
                       'PackingCategory', 'PickUpArea']]

In [ ]:
dfAllData.to_excel(BASE_DIR / outputDirectory / fileNameAllData)

### Plocklista

In [ ]:
def getFirstCellForNamedRange(workbook, sheet, range_name):
    cellNameList = []
    firstCellTuple = ()
    if range_name in workbook.defined_names:
        cellNameList = [(s, c) for s, c in workbook.defined_names[range_name].destinations if (s == sheet.title)]
    if cellNameList:
        firstCellTuple = cellNameList[0]
    return firstCellTuple[1]

In [ ]:
def fillCellForAttribute(rangeName, workbook, template, sheet, row_offset, value):
    templateCellNumber = getFirstCellForNamedRange(workbook, template, rangeName)
    targetCellNumber = template[templateCellNumber].offset(row=row_offset).coordinate
    sheet[targetCellNumber] = value

In [ ]:
def getMergedCellsOrNone(sheet, row, column):
    cell = sheet.cell(row, column)
    for mergedCell in sheet.merged_cells.ranges:
        if (cell.coordinate in mergedCell):
            return mergedCell
    return None

In [ ]:
def insertRowAndCopyFormat(template, sheet, source_row, target_row):
    sheet.insert_rows(idx=target_row)
    sheet.row_dimensions[target_row].height = template.row_dimensions[source_row].height
    for col in range(1, 11):
        sheet.cell(row=target_row, column=col)._style = template.cell(row=source_row, column=col)._style
        mergedCells = getMergedCellsOrNone(template, source_row, col)
        if mergedCells is not None and col == mergedCells.bounds[0]:
            mergedCellsCopy = copy(mergedCells)
            if target_row > source_row:
                mergedCellsCopy.shift(row_shift=target_row-source_row)
                sheet.merge_cells(range_string=(mergedCellsCopy.coord))

In [ ]:
Path.unlink(BASE_DIR / outputDirectory / fileNamePackList, missing_ok=True)
shutil.copyfile(BASE_DIR / templateDirectory / fileNamePackListTemplate, 
                BASE_DIR / outputDirectory / fileNamePackList)

In [ ]:
workbook = load_workbook(filename=BASE_DIR / outputDirectory / fileNamePackList)
template_sheet = workbook['template']

In [ ]:
dfAllData = dfAllData.sort_values(by=['PickUpDate', 'PickUpTime'])

In [ ]:
dfGroupedByOrderNumber = dfAllData.groupby(by=['PickUpDate', 'PickUpTime', 'OrderNumber'])

In [ ]:
articleCell = getFirstCellForNamedRange(workbook, template_sheet, 'ArticleNumber')
headerRow = template_sheet[articleCell].row

for index, group in dfGroupedByOrderNumber:
    new_sheet = workbook.copy_worksheet(template_sheet)
    order_number = group.iloc[0]['OrderNumber']
    new_sheet.title = 'PO_' + str(order_number)
    for rangeName in ['FullName', 'OrderTotalAmount', 'CustomerNote', 'OrderNumber', 'Telephone', 'PickUpDate', 'PickUpTime']:
        fieldValue = group.iloc[0][rangeName]
        fillCellForAttribute(rangeName, workbook, template_sheet, new_sheet, 0, fieldValue)
    new_sheet.delete_rows(idx=headerRow, amount=2)
    dfGroupedByPickUpArea = group.groupby('PickUpArea')
    i = 0
    d = 0
    for area, subGroup in dfGroupedByPickUpArea:
        for sub_index, row in subGroup.iterrows():
            # sub_index is the (arbitrary) row number and cannot be used instead of i!
            insertRowAndCopyFormat(template_sheet, new_sheet, headerRow, headerRow+i)
            for rangeName in ['ArticleNumber', 'Quantity', 'ProductName', 'ItemCost', 'PickUpArea']:
                fieldValue = row[rangeName]
                fillCellForAttribute(rangeName, workbook, template_sheet, new_sheet, i, fieldValue)
            i+=1
            fillCellForAttribute('RowNumber', workbook, template_sheet, new_sheet, i-1, i-d)
        insertRowAndCopyFormat(template_sheet, new_sheet, headerRow+1, headerRow+i)
        i+=1
        d+=1
    for j in range(0,10):
        offset = headerRow+i
        new_sheet.row_dimensions[offset+j].height = template_sheet.row_dimensions[headerRow+2+j].height
    fillCellForAttribute('NumberOfRows', workbook, template_sheet, new_sheet, 0, 'Ant.Orderrader ' + str(i-d))

In [ ]:
workbook.remove(template_sheet)

In [ ]:
workbook.save(filename=BASE_DIR / outputDirectory / fileNamePackList)

### Hämtlista

In [ ]:
weekDayDict = {
0: 'Mån',
1: 'Tis',
2: 'Ons',
3: 'Tor',
4: 'Fre',
5: 'Lör',
6: 'Sön'}

In [ ]:
Path.unlink(BASE_DIR / outputDirectory / fileNamePickUpList, missing_ok=True)
shutil.copyfile(BASE_DIR / templateDirectory / fileNamePickUpListTemplate,
                BASE_DIR / outputDirectory / fileNamePickUpList)

In [ ]:
workbook = load_workbook(filename=BASE_DIR / outputDirectory / fileNamePickUpList)
template_sheet = workbook['template']

In [ ]:
dfGroupedByPickUpDate = dfAllData.drop_duplicates(subset=['OrderNumber']).groupby('PickUpDate')

In [ ]:
pickUpTimeCell = getFirstCellForNamedRange(workbook, template_sheet, 'PickUpTime')
orderNumberCol = template_sheet[pickUpTimeCell].column

for ts, group in dfGroupedByPickUpDate:
    new_sheet = workbook.copy_worksheet(template_sheet)
    new_sheet.title = 'Hämtas ' + str(ts.date())
    fillCellForAttribute('Weekday', workbook, template_sheet, new_sheet, 0, weekDayDict[ts.weekday()])
    fillCellForAttribute('PickUpDate', workbook, template_sheet, new_sheet, 0, ts.date())
    fillCellForAttribute('CreationDateTime', workbook, template_sheet, new_sheet, 0, str(datetime.now()))
    headerRow = template_sheet[pickUpTimeCell].row
    new_sheet.delete_rows(idx=headerRow, amount=2)
    dfGroupedByPickUpTime = group.groupby('PickUpTime')
    scheduledTimes = dfSchedule[dfSchedule['Date']==ts.date()]['Tid']
    i = 0
    d = 0
    for time in scheduledTimes:
        insertRowAndCopyFormat(template_sheet, new_sheet, headerRow, headerRow+i)
        fillCellForAttribute('PickUpTime', workbook, template_sheet, new_sheet, i, time)
        i+=1
        d+=1
        if time in dfGroupedByPickUpTime.indices:
            subGroup = dfGroupedByPickUpTime.get_group(time)
            for index, row in subGroup.iterrows():
                insertRowAndCopyFormat(template_sheet, new_sheet, headerRow+1, headerRow+i)
                for rangeName in ['OrderNumber', 'FullName', 'Telephone', 'OrderTotalAmount', 'CustomerNote']:
                    fieldValue = row[rangeName]
                    fillCellForAttribute(rangeName, workbook, template_sheet, new_sheet, i-1, fieldValue)
                i+=1
                fillCellForAttribute('RowNumber', workbook, template_sheet, new_sheet, i-2, i-d)
        else:
            insertRowAndCopyFormat(template_sheet, new_sheet, headerRow+1, headerRow+i)
            i+=1
            fillCellForAttribute('RowNumber', workbook, template_sheet, new_sheet, i-2, i-d)

In [ ]:
workbook.remove(template_sheet)

In [ ]:
workbook.save(filename=BASE_DIR / outputDirectory / fileNamePickUpList)

### Semlelista

In [ ]:
Path.unlink(BASE_DIR / outputDirectory / fileNameSemleList, missing_ok=True)
shutil.copyfile(BASE_DIR / templateDirectory / fileNameSemleListTemplate, 
                BASE_DIR / outputDirectory / fileNameSemleList)

In [ ]:
workbook = load_workbook(filename=BASE_DIR / outputDirectory / fileNameSemleList)
template_sheet = workbook['template']

In [ ]:
articleNumberSemla = '#H_11#'
dfOrdersSemla = dfAllData[dfAllData['ArticleNumber'] == articleNumberSemla]
dfOrdersSemla = dfOrdersSemla[['ArticleNumber', 'ProductName', 'Quantity', 'OrderNumber', 'FullName', 'Email', 'PickUpDate', 'PickUpTime']]
dfOrdersSemla['PickUpDateTime'] = [datetime.combine(d, t) for d, t in zip(dfOrdersSemla['PickUpDate'], dfOrdersSemla['PickUpTime'])]
dfOrdersSemla['PickUpHour']  = dfOrdersSemla['PickUpDateTime'].dt.hour

In [ ]:
dfGroupedByPickUpDate = dfOrdersSemla.groupby(['PickUpDate'])

In [ ]:
orderNumberCell = getFirstCellForNamedRange(workbook, template_sheet, 'OrderNumber')
headerRow = template_sheet[orderNumberCell].row

new_sheet = workbook.copy_worksheet(template_sheet)
new_sheet.title = 'Semla-Lista'

for rangeName in ['ArticleNumber', 'ProductName']:
    fieldValue = dfOrdersSemla.iloc[0][rangeName]
    fillCellForAttribute(rangeName, workbook, template_sheet, new_sheet, 0, fieldValue)
new_sheet.delete_rows(idx=headerRow, amount=2)

i = 0

for day, group in dfGroupedByPickUpDate:
    dfGroupedByPickUpArea = group.groupby('PickUpHour')
    
    for hour, subGroup in dfGroupedByPickUpArea:
        for sub_index, row in subGroup.iterrows():
            insertRowAndCopyFormat(template_sheet, new_sheet, headerRow, headerRow+i)
            for rangeName in ['Quantity', 'OrderNumber', 'FullName', 'Email', 'PickUpDate', 'PickUpTime']:
                fieldValue = row[rangeName]
                fillCellForAttribute(rangeName, workbook, template_sheet, new_sheet, i, fieldValue)
            i+=1
        insertRowAndCopyFormat(template_sheet, new_sheet, headerRow+1, headerRow+i)
        fillCellForAttribute('Day', workbook, template_sheet, new_sheet, i-1, day.strftime('%d.%m'))
        timeIntervalString = str(hour) + ':00 - ' + str(hour+1) + ':00'
        fillCellForAttribute('Hour', workbook, template_sheet, new_sheet, i-1, timeIntervalString)
        fillCellForAttribute('NumberOfItemsName', workbook, template_sheet, new_sheet, i-1, 'Antal:')
        numberOfItemsString = subGroup['Quantity'].sum()
        fillCellForAttribute('NumberOfItems', workbook, template_sheet, new_sheet, i-1, numberOfItemsString)
        i+=2
        
    for j in range(0,10):
        offset = headerRow+i
        new_sheet.row_dimensions[offset+j].height = template_sheet.row_dimensions[headerRow+2+j].height

In [ ]:
workbook.remove(template_sheet)

In [ ]:
workbook.save(filename=BASE_DIR / outputDirectory / fileNameSemleList)

### Masterlista

In [ ]:
dfMasterList = dfAllData[['OrderNumber', 'FullName', 'Email', 'OrderTotalAmount', 'OrderDate', 
              'Telephone', 'CustomerNote', 'Info', 'PickUpDate', 'PickUpTime']]

In [ ]:
dfMasterList.columns = ['Order Number', 'Full Name (Billing)', 'Email (Billing)', 'Order Total Amount', 'Order Date', 
              'Telephone', 'Customer Note', 'Info ok', 'Hämtningsdag', 'Hämtning Tid']

In [ ]:
dfMasterList = dfMasterList.drop_duplicates()

In [ ]:
dfMasterList.to_excel(BASE_DIR / outputDirectory / fileNameMasterList)

In [ ]:
# tdr = pd.timedelta_range(start=str(dfSchedule['Tid'].min()), 
#                    end=str(dfSchedule['Tid'].max()), 
#                    freq='1H')